# Instagram Data extraction and regression tutorial

face API sample:https://azure.microsoft.com/en-au/services/cognitive-services/face/
IBM tone emotion :https://tone-analyzer-demo.ng.bluemix.net/

instagram scrap tool:
https://phantombuster.com/api-store   Instagram Posts Extractor 

Homework: https://docs.google.com/spreadsheets/d/1nBP_FBGgcPh4xCsG3kkj-phudhXqePbFjjOZYSpKENE/edit?fbclid=IwAR1Fr2MJJFS9WN4Zyr1xQHw_YB2LVUxTIGOtoVYtSHhJC92fXlAvYZAGJ6w#gid=0

In [0]:
import io
import cv2 
import glob
import pandas as pd
import numpy as np
from google.colab import files
import os
import requests
from PIL import Image
import requests
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab import files
import urllib
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import requests
from PIL import Image

Some help functions

In [0]:

def dummies_dic(column_name,df,drop_number,drop_score):
  column=df[column_name]
  my_list=[]
  for i in column:
    tmp=[]
    i=str(i)
    for j in  eval(i):
      tmp.append(j)
    my_list.append(tmp)
  new_column=pd.Series(my_list)
  vectorizer=MultiLabelBinarizer()
  df_2 = pd.DataFrame(vectorizer.fit_transform(new_column),index=df.index,columns=column_name+'_' +vectorizer.classes_)
  tmp=[]
  delet_colume_name=[]
  for column in df_2.columns:
    if np.count_nonzero(df_2[column]) <drop_number:
      delet_colume_name.append(column)
      del df_2[column]
  column=df[column_name]
  for i in column:
    i=str(i)
    tmp.append(float(len(eval(i))))
  new_column=tmp
  #print(new_column)
  #df_3=pd.DataFrame(new_column,index=df.index, columns=['The sum of '+column_name ])
  result = df.join(df_2, how='outer')
  #result = result_1.join(df_3, how='outer')
  column=result[column_name]
  count=0

  for i in column:
    i=str(i)
    for j in eval(i):
      lable_name=str(column_name+'_' +j)
      #print(i)
      score=eval(i)[j]
      #print(score)
      if lable_name in delet_colume_name:
        continue

      if score <drop_score:
        score=0

      result.loc[count,lable_name]=score
    count=count+1

  return result

# Data download

In [0]:
#updata the data.csv from the instagram scrap
files.upload()

In [0]:
row_number=0#you can change the number if you want to skip download the top n row
source = pd.read_csv('result.csv')# write you file name 


for i in range(len(source)):
  if i < row_number:
    continue
  if source.loc[i,'type']=='Video':
    continue
  link=source.loc[i,'imgUrl']
  name=str(i)+".jpg"
  try:
    img_data = requests.get(link ).content
  except:
    continue

  dirname = os.path.dirname(name)
  
  with open(name, 'wb') as handler:
    handler.write(img_data)

  try:
    files.download(name)
  except:
    print(name)
    continue

14.jpg
23.jpg


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Image processing

apply the free acount:https://azure.microsoft.com/en-au/services/cognitive-services/face/

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import glob

list_of_files = glob.glob('/content/gdrive/My Drive/Marketing-AI-Lab/pencils of promise/*.*')#write you own path from the google drive

replace you microsoft key in here in Third lines 

In [0]:
def face_detect(image):
  face_api_url = 'https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect'
  
  headers = { 'Content-Type': 'application/octet-stream', 'Ocp-Apim-Subscription-Key': '0e76fc77353e482dab9e1c1b42361bc3' }#replace you own key in here
  opened_file = open(image, 'rb')
  #print(opened_file)
  params = urllib.parse.urlencode({
      'returnFaceId': 'true',
      'returnFaceLandmarks': 'false',
      'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
  })

  response = requests.post(face_api_url, params=params, headers=headers, data=opened_file)
  #print(response)
  result=response.json()
  return result

In [0]:
import time
smile=[]
gender=[]
age=[]
facialHair=[]
glasses=[]
emotion=[]
blur=[]
exposure=[]
noise=[]
makeup=[]
accessories=[]
occlusion=[]
hair=[]
picture_name=[]
for image in list_of_files:
  time.sleep(3)
  try:
    result=face_detect(image)
    if result==[]:
          name=image.split('/')
          name_1=name[-1].split('.')
          picture_name.append(name_1)
          smile.append([])
          gender.append([])
          age.append([])
          facialHair.append([])
          glasses.append([])
          emotion.append([])
          blur.append([])
          exposure.append([])
          noise.append([])
          makeup.append([])
          accessories.append([])
          occlusion.append([])
          hair.append([])
    for i in result:
        try:
          name=image.split('/')
          name_1=name[-1].split('.')
          picture_name.append(name_1[0])
        except:
          picture_name.append([])
        try:
          smile.append(i['faceAttributes']['smile'])
        except:
          smile.append([])
        try:
          gender.append(i['faceAttributes']['gender'])
        except:
          gender.append([])
        try:
          age.append(i['faceAttributes']['age'])
        except:
          age.append([])
        try:
          facialHair.append(i['faceAttributes']['facialHair'])
        except:
          facialHair.append([])
        try:
          glasses.append(i['faceAttributes']['glasses'])
        except:
          glasses.append([])
        try:
          emotion.append(i['faceAttributes']['emotion'])
        except:
          emotion.append([])
        try:
          blur.append(i['faceAttributes']['blur'])
        except:
          blur.append([])
        try:
          exposure.append(i['faceAttributes']['exposure'])
        except:
          exposure.append([])
        try:
          noise.append(i['faceAttributes']['noise'])
        except:
          noise.append([])
        try:        
          makeup.append(i['faceAttributes']['makeup'])
        except:
          makeup.append([])
        try: 
          accessories.append(i['faceAttributes']['accessories'])
        except:
          accessories.append([])
        try:                  
          occlusion.append(i['faceAttributes']['occlusion'])
        except:
          occlusion.append([])
        try:
          hair.append(i['faceAttributes']['hair'])
        except:
          hair.append([])


  except:
          print(image)
          print(result)
          continue


new_csv={'picture_name':picture_name,'smile':smile,'gender':gender,'age':age,'facialHair':facialHair,'glasses':glasses,'emotion':emotion,'blur':blur,'exposure':exposure,'noise':noise,'makeup':makeup,'accessories':accessories,'occlusion':occlusion,'hair':hair}
Face_result = pd.DataFrame(new_csv)


In [0]:
Face_result 

,picture_name,smile,gender,age,facialHair,glasses,emotion,blur,exposure,noise,makeup,accessories,occlusion,hair
0,"[1405, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[]
1,1409,0.415,female,6,"{'moustache': 0.0, 'beard': 0.0, 'sideburns': ...",NoGlasses,"{'anger': 0.0, 'contempt': 0.005, 'disgust': 0...","{'blurLevel': 'medium', 'value': 0.43}","{'exposureLevel': 'goodExposure', 'value': 0.6}","{'noiseLevel': 'low', 'value': 0.18}","{'eyeMakeup': True, 'lipMakeup': True}",[],"{'foreheadOccluded': False, 'eyeOccluded': Fal...","{'bald': 0.24, 'invisible': False, 'hairColor'..."
2,1409,0.027,female,7,"{'moustache': 0.0, 'beard': 0.0, 'sideburns': ...",NoGlasses,"{'anger': 0.0, 'contempt': 0.001, 'disgust': 0...","{'blurLevel': 'medium', 'value': 0.31}","{'exposureLevel': 'overExposure', 'value': 0.82}","{'noiseLevel': 'low', 'value': 0.0}","{'eyeMakeup': False, 'lipMakeup': True}",[],"{'foreheadOccluded': False, 'eyeOccluded': Fal...","{'bald': 0.32, 'invisible': False, 'hairColor'..."
3,"[1408, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[]
4,"[1407, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[]
5,"[1406, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[]
6,"[1410, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[]
7,"[1403, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[]
8,"[1402, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[]
9,1404,0.95,male,6,"{'moustache': 0.0, 'beard': 0.0, 'sideburns': ...",NoGlasses,"{'anger': 0.0, 'contempt': 0.001, 'disgust': 0...","{'blurLevel': 'low', 'value': 0.0}","{'exposureLevel': 'goodExposure', 'value': 0.51}","{'noiseLevel': 'low', 'value': 0.0}","{'eyeMakeup': False, 'lipMakeup': False}",[],"{'foreheadOccluded': False, 'eyeOccluded': Fal...","{'bald': 0.15, 'invisible': False, 'hairColor'..."


In [0]:
data=dummies_dic('emotion',Face_result,0,0)

In [0]:
data

,picture_name,smile,gender,age,facialHair,glasses,emotion,blur,exposure,noise,makeup,accessories,occlusion,hair,emotion_anger,emotion_contempt,emotion_disgust,emotion_fear,emotion_happiness,emotion_neutral,emotion_sadness,emotion_surprise
0,"[1405, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[],0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1,1409,0.415,female,6,"{'moustache': 0.0, 'beard': 0.0, 'sideburns': ...",NoGlasses,"{'anger': 0.0, 'contempt': 0.005, 'disgust': 0...","{'blurLevel': 'medium', 'value': 0.43}","{'exposureLevel': 'goodExposure', 'value': 0.6}","{'noiseLevel': 'low', 'value': 0.18}","{'eyeMakeup': True, 'lipMakeup': True}",[],"{'foreheadOccluded': False, 'eyeOccluded': Fal...","{'bald': 0.24, 'invisible': False, 'hairColor'...",0.000,0.005,0.000,0.000,0.415,0.577,0.001,0.002
2,1409,0.027,female,7,"{'moustache': 0.0, 'beard': 0.0, 'sideburns': ...",NoGlasses,"{'anger': 0.0, 'contempt': 0.001, 'disgust': 0...","{'blurLevel': 'medium', 'value': 0.31}","{'exposureLevel': 'overExposure', 'value': 0.82}","{'noiseLevel': 'low', 'value': 0.0}","{'eyeMakeup': False, 'lipMakeup': True}",[],"{'foreheadOccluded': False, 'eyeOccluded': Fal...","{'bald': 0.32, 'invisible': False, 'hairColor'...",0.000,0.001,0.000,0.000,0.027,0.971,0.001,0.000
3,"[1408, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[],0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,"[1407, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[],0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,"[1406, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[],0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
6,"[1410, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[],0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
7,"[1403, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[],0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
8,"[1402, jpg]",[],[],[],[],[],[],[],[],[],[],[],[],[],0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
9,1404,0.95,male,6,"{'moustache': 0.0, 'beard': 0.0, 'sideburns': ...",NoGlasses,"{'anger': 0.0, 'contempt': 0.001, 'disgust': 0...","{'blurLevel': 'low', 'value': 0.0}","{'exposureLevel': 'goodExposure', 'value': 0.51}","{'noiseLevel': 'low', 'value': 0.0}","{'eyeMakeup': False, 'lipMakeup': False}",[],"{'foreheadOccluded': False, 'eyeOccluded': Fal...","{'bald': 0.15, 'invisible': False, 'hairColor'...",0.000,0.001,0.000,0.000,0.950,0.036,0.013,0.000


Process mutipual face

In [0]:
 
data_emotion=data
data_emotion=data[['emotion_anger',
       'emotion_contempt', 'emotion_disgust', 'emotion_fear',
       'emotion_happiness', 'emotion_neutral', 'emotion_sadness',
       'emotion_surprise']]


import numpy as np
emotion_list=['emotion_anger',
       'emotion_contempt', 'emotion_disgust', 'emotion_fear',
       'emotion_happiness', 'emotion_neutral', 'emotion_sadness',
       'emotion_surprise']
for index, row in data_emotion.iterrows():
  max_number=row.values.max()
  if max_number==0:
    continue
  max_index=np.argmax(row.values)
  for i in range(len(emotion_list)):
    if i == max_index:
      data.loc[index,emotion_list[i]]=1
    else:
      data.loc[index,emotion_list[i]]=0
acount=0
for i in data['picture_name']:
  i=str(i)
  if i[0]=="[":
    data.loc[acount,'picture_name'] =eval(i)[0]
  acount=acount+1

In [0]:
data.dropna(subset=['picture_name'],inplace=True)
data_sum_emotion=data[['picture_name','emotion_anger',
       'emotion_contempt', 'emotion_disgust', 'emotion_fear',
       'emotion_happiness', 'emotion_neutral', 'emotion_sadness',
       'emotion_surprise']]
data_emotion=data_sum_emotion.groupby(['picture_name']).sum().reset_index()
d = {'face_count': data_sum_emotion['picture_name'], 'col2':  data_sum_emotion['picture_name']}
df = pd.DataFrame(data=d)                                      
data_emotion_2=df.groupby(['col2']).count().reset_index()
data_emotion_facenumber=data_emotion.merge(data_emotion_2, left_on='picture_name', right_on='col2',how='inner',suffixes=(False, False))

In [0]:
data_emotion_facenumber

,picture_name,emotion_anger,emotion_contempt,emotion_disgust,emotion_fear,emotion_happiness,emotion_neutral,emotion_sadness,emotion_surprise,col2,face_count
0,1136,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1136,3
1,1137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1137,1
2,1138,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1138,1
3,1139,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1139,1
4,1140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1140,1
5,1141,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1141,1
6,1142,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1142,1
7,1143,0.0,0.0,0.0,0.0,2.0,3.0,1.0,0.0,1143,6
8,1144,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1144,4
9,1145,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1145,1


Save this data

In [0]:
data_emotion_facenumber.to_csv('face_emotion.csv')

# Text processing

In [0]:
#upload the json file from the instagram scrap
uploaded = files.upload()

In [0]:
!pip install ibm-watson

     |████████████████████████████████| 276kB 2.8MB/s 
     |████████████████████████████████| 204kB 46.8MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-3.4.1-cp36-none-any.whl size=275905 sha256=5f3d2fb2adf92d1f75cdfb3cdad294f3232bc7df1cddb2ff5f9455dd0bde3ae0
  Stored in directory: /root/.cache/pip/wheels/20/2d/29/4beb79a6cd0ba7a443ceee2366eec4ffe88a1476c1125d3c74
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-0.5.1-cp36-none-any.whl size=15841 sha256=25a8e3b93c55a58f8925be9cbca42d16694665b4de4dd4f1f63d0fba9fb4c7fa
  Stored in directory: /root/.cache/pip/wheels/4d/42/cd/c4cdd4cc793679714e817ecfcfaa77dfbacf2af34c7d27ffd0
Successfully built ibm-watson ibm-cloud-sdk-core


Replace IBM key in 6th lines

In [0]:
import re
import json
from ibm_watson import ToneAnalyzerV3

tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    iam_apikey='iSh44vKuTUb1vSZdZEmusfi4L9bU9VCoE5o1VvfiMLDE',#replace key in here 
    url='https://gateway.watsonplatform.net/tone-analyzer/api'
)

output=[]

with open('result.json') as f:
    s=json.load(f)
    index=0
    for index in range(len(s)):
        if s[index]['type']!='Photo':
            continue
        try:
            text=s[index]['description'].replace('#','').replace('@','').encode('ascii', 'ignore').decode('ascii')
        except:
            continue
        #print(index,text)
        tone_analysis = tone_analyzer.tone(
            {'text': text},
            content_type='application/json').get_result()
        element={'id':index,'text':text,'response':tone_analysis}
        output.append(element)
with open('text_emotion.json','w') as outfile:
    json.dump(output,outfile)

In [0]:
def formatToneResponse(toneJsonName):
    with open(toneJsonName+'.json') as jsonfile:
        l=[]
        s=json.load(jsonfile)
        #print(s)
        for i in range(len(s)):
            element={'id':s[i]['id'],'description':s[i]['text'],'tone':s[i]['response']['document_tone']['tones']}
            l.append(element)
    df=pd.DataFrame(columns=['id','description','anger','fear','joy','sadness','analytical','confident','tentative'])
    toneDict={'anger':2,'fear':3,'joy':4,'sadness':5,'analytical':6,'confident':7,'tentative':8}
    for ele in l:
        print(ele)
        row={'id':ele['id'],'description':ele['description']}
        for subele in ele['tone']:
            #print(subele['tone_id'])
            row[subele['tone_id']]=subele['score']
        df=df.append(row,ignore_index=True)
    df.to_csv(toneJsonName+'.csv',index=False)

In [0]:
import pandas as pd
formatToneResponse('text_emotion')

{'id': 0, 'description': 'We\'ve got a super special surprise for your Friday! We\'re excited to introduce you to our Laos Co-Country Director, Ya. Today he\'s sharing one of his favourite PoP memories: "I traveled with the team on a new community-scouting trip for our 2019 school builds. The ride was enjoyable, as we drove through green fields and thick jungle. I could see birds flying all around and the hills and valleys one after another as far as the horizon. When we arrived, the existing school was so dilapidated. It had a tin roof and bamboo walls with termites. The structure looked like it was tilting and could fall down soon. The students seemed to not care about the condition of their school. This was their first school and the only option they had. The kids seemed to enjoy school more than anything else. They had smiles on their faces and in every classroom. They paid attention when their teacher was talking. School is a place for them to learn new things that their parents w

In [0]:
data_text=pd.read_csv('text_emotion.csv')


In [0]:
data_text

,id,description,anger,fear,joy,sadness,analytical,confident,tentative
0,0,We've got a super special surprise for your Fr...,NaN,NaN,0.661455,0.502281,NaN,NaN,0.617927
1,3,Thank you billboard for the love! Were excited...,NaN,NaN,0.962007,NaN,0.619540,NaN,NaN
2,5,"""Education helps us learn more and be someone ...",NaN,NaN,0.725988,NaN,NaN,NaN,0.689515
3,6,"PoP's Chief Impact Officer, lengleyoung, moved...",NaN,NaN,0.703334,NaN,NaN,0.500192,NaN
4,7,"PoP's Chief Impact Officer, lengleyoung, moved...",NaN,NaN,0.703334,NaN,NaN,0.500192,NaN
5,8,"PoP's Chief Impact Officer, lengleyoung, moved...",NaN,NaN,0.703334,NaN,NaN,0.500192,NaN
6,9,"PoP's Chief Impact Officer, lengleyoung, moved...",NaN,NaN,0.703334,NaN,NaN,0.500192,NaN
7,10,Don't we all love our devices? They're even be...,NaN,NaN,0.573401,NaN,NaN,0.562307,NaN
8,11,"Cover, Title, Author. Let's continue to advanc...",NaN,NaN,NaN,NaN,0.734627,NaN,NaN
9,12,Pay close attention: dreams are being created!...,NaN,NaN,0.632424,NaN,0.968356,NaN,NaN


Merge_face emotion data and text emotion data

In [0]:
uploaded = files.upload()

In [0]:

data_new=data_text.merge(data_emotion_facenumber, left_on=data_text.id.astype(str),right_on=data_emotion_facenumber.picture_name.astype(str),how='inner',suffixes=(False, False))



In [0]:
data_new

,key_0,id,description,anger,fear,joy,sadness,analytical,confident,tentative,picture_name,emotion_anger,emotion_contempt,emotion_disgust,emotion_fear,emotion_happiness,emotion_neutral,emotion_sadness,emotion_surprise,col2,face_count
0,225,225,Meet timothysykes. 1 year ago he donated $1 mi...,NaN,NaN,0.679337,NaN,NaN,NaN,NaN,225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225,1
1,226,226,Meet timothysykes. 1 year ago he donated $1 mi...,NaN,NaN,0.679337,NaN,NaN,NaN,NaN,226,0.0,0.0,0.0,0.0,1.0,7.0,1.0,0.0,226,9
2,228,228,A huge thanks to the incredible 2017 Pencils o...,NaN,NaN,0.909215,NaN,NaN,NaN,NaN,228,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,228,2
3,229,229,A huge thanks to the incredible 2017 Pencils o...,NaN,NaN,0.909215,NaN,NaN,NaN,NaN,229,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,229,2
4,264,264,"It's September, which means back to school for...",NaN,NaN,NaN,NaN,0.760380,0.530214,NaN,264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,264,1
5,265,265,"It's September, which means back to school for...",NaN,NaN,NaN,NaN,0.760380,0.530214,NaN,265,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,265,1
6,268,268,"Check out LiteracyDay2017 to meet Tadam, age 1...",NaN,NaN,0.704959,NaN,NaN,NaN,0.829677,268,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,268,1
7,269,269,"Check out LiteracyDay2017 to meet Tadam, age 1...",NaN,NaN,0.704959,NaN,NaN,NaN,0.829677,269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,269,1
8,278,278,"Celebrating Freeman Gobah, our stellar Ghana C...",NaN,NaN,0.664676,NaN,0.619540,0.838624,NaN,278,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,278,1
9,279,279,"Celebrating Freeman Gobah, our stellar Ghana C...",NaN,NaN,0.664676,NaN,0.619540,0.838624,NaN,279,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,279,3


In [0]:
#Upload your data.csv from the instram Instagram Posts Extractor
uploaded = files.upload()

In [0]:
Orgional_data=pd.read_csv('result.csv')#change the file name to yours 
data_new.drop(columns=['description','key_0'],inplace=True)
data=data_new.merge(Orgional_data, left_on=data_new.id.astype(int),right_index=True,how='inner',suffixes=(False, False))

In [0]:
data

,key_0,id,anger,fear,joy,sadness,analytical,confident,tentative,picture_name,emotion_anger,emotion_contempt,emotion_disgust,emotion_fear,emotion_happiness,emotion_neutral,emotion_sadness,emotion_surprise,col2,face_count,postUrl,description,commentCount,likeCount,pubDate,type,caption,profileUrl,username,fullName,imgUrl,postId,timestamp,query,videoUrl,viewCount,location,locationId,isSidecar,taggedFullName1,taggedUsername1,taggedFullName2,taggedUsername2,taggedFullName3,taggedUsername3,taggedFullName4,taggedUsername4,taggedFullName5,taggedUsername5,taggedFullName6,taggedUsername6,taggedFullName7,taggedUsername7,taggedFullName8,taggedUsername8,taggedFullName9,taggedUsername9,taggedFullName10,taggedUsername10,taggedFullName11,taggedUsername11,taggedFullName12,taggedUsername12,taggedFullName13,taggedUsername13,taggedFullName14,taggedUsername14,taggedFullName15,taggedUsername15,taggedFullName16,taggedUsername16,taggedFullName17,taggedUsername17,taggedFullName18,taggedUsername18,taggedFullName19,taggedUsername19,taggedFullName20,taggedUsername20
0,225,225,NaN,NaN,0.679337,NaN,NaN,NaN,NaN,225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225,1,https://www.instagram.com/p/BbSRzr9BTeU/,Meet @timothysykes. 1 year ago he donated $1 m...,23.0,1980,2017-11-09T19:05:49.000Z,Photo,NaN,https://www.instagram.com/pencilsofpromise/,pencilsofpromise,Pencils of Promise,https://scontent-lga3-1.cdninstagram.com/vp/af...,1644454402008161562,2019-09-30T15:07:56.668Z,https://www.instagram.com/pencilsofpromise,NaN,NaN,NaN,NaN,True,Timothy Sykes,timothysykes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,226,226,NaN,NaN,0.679337,NaN,NaN,NaN,NaN,226,0.0,0.0,0.0,0.0,1.0,7.0,1.0,0.0,226,9,https://www.instagram.com/p/BbSRzr9BTeU/,Meet @timothysykes. 1 year ago he donated $1 m...,23.0,1980,2017-11-09T19:05:49.000Z,Photo,NaN,https://www.instagram.com/pencilsofpromise/,pencilsofpromise,Pencils of Promise,https://scontent-lga3-1.cdninstagram.com/vp/82...,1644454419666176038,2019-09-30T15:07:56.668Z,https://www.instagram.com/pencilsofpromise,NaN,NaN,NaN,NaN,True,Timothy Sykes,timothysykes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,228,228,NaN,NaN,0.909215,NaN,NaN,NaN,NaN,228,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,228,2,https://www.instagram.com/p/BbPl14oBbBQ/,A huge thanks to the incredible 2017 Pencils o...,24.0,2282,2017-11-08T18:03:10.000Z,Photo,NaN,https://www.instagram.com/pencilsofpromise/,pencilsofpromise,Pencils of Promise,https://scontent-lga3-1.cdninstagram.com/vp/45...,1643698073006040249,2019-09-30T15:07:56.668Z,https://www.instagram.com/pencilsofpromise,NaN,NaN,TCS New York City Marathon,7.163811e+10,True,Finn Harries,finnharries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,229,229,NaN,NaN,0.909215,NaN,NaN,NaN,NaN,229,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,229,2,https://www.instagram.com/p/BbPl14oBbBQ/,A huge thanks to the incredible 2017 Pencils o...,24.0,2282,2017-11-08T18:03:10.000Z,Photo,NaN,https://www.instagram.com/pencilsofpromise/,pencilsofpromise,Pencils of Promise,https://scontent-lga3-1.cdninstagram.com/vp/83...,1643698076864873103,2019-09-30T15:07:56.668Z,https://www.instagram.com/pencilsofpromise,NaN,NaN,TCS New York City Marathon,7.163811e+10,True,Finn Harries,finnharries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,264,264,NaN,NaN,NaN,NaN,0.760380,0.530214,NaN,264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,264,1,https://www.instagram.com/p/BY_G75hB6dg/,"It's September, which means back to school for...",5.0,716,2017-09-13T15:22:28.000Z,Photo,NaN,https://www.instagram.com/pencilsofpromise/,pencilsofpromise,Pencils of Promise,https://scontent-lga3-1.

In [0]:
data.to_csv('Final_raw_data.csv')

# Regression part

In [0]:
#uploaded = files.upload()
#data=pd.read_csv('new.csv')

In [0]:
columns=['likeCount','anger','fear',	'joy',	'sadness',	'analytical'	,'confident'	,'tentative','emotion_anger'	,'emotion_contempt'	,'emotion_disgust'	,'emotion_fear',	'emotion_happiness',	'emotion_neutral'	,'emotion_sadness'	,'emotion_surprise']
regression_data=data[columns]


In [0]:
regression_data

,likeCount,anger,fear,joy,sadness,analytical,confident,tentative,emotion_anger,emotion_contempt,emotion_disgust,emotion_fear,emotion_happiness,emotion_neutral,emotion_sadness,emotion_surprise
0,1980,NaN,NaN,0.679337,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1980,NaN,NaN,0.679337,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,7.0,1.0,0.0
2,2282,NaN,NaN,0.909215,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
3,2282,NaN,NaN,0.909215,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,716,NaN,NaN,NaN,NaN,0.760380,0.530214,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,716,NaN,NaN,NaN,NaN,0.760380,0.530214,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,956,NaN,NaN,0.704959,NaN,NaN,NaN,0.829677,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,956,NaN,NaN,0.704959,NaN,NaN,NaN,0.829677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,858,NaN,NaN,0.664676,NaN,0.619540,0.838624,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,858,NaN,NaN,0.664676,NaN,0.619540,0.838624,NaN,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0


In [0]:
data.replace(0, np.nan, inplace=True)
data.describe()

,key_0,id,anger,fear,joy,sadness,analytical,confident,tentative,emotion_anger,emotion_contempt,emotion_disgust,emotion_fear,emotion_happiness,emotion_neutral,emotion_sadness,emotion_surprise,face_count,commentCount,likeCount,caption,postId,videoUrl,viewCount,locationId,taggedFullName4,taggedUsername4,taggedFullName5,taggedUsername5,taggedFullName6,taggedUsername6,taggedFullName7,taggedUsername7,taggedFullName8,taggedUsername8,taggedFullName9,taggedUsername9,taggedFullName10,taggedUsername10,taggedFullName11,taggedUsername11,taggedFullName12,taggedUsername12,taggedFullName13,taggedUsername13,taggedFullName14,taggedUsername14,taggedFullName15,taggedUsername15,taggedFullName16,taggedUsername16,taggedFullName17,taggedUsername17,taggedFullName18,taggedUsername18,taggedFullName19,taggedUsername19,taggedFullName20,taggedUsername20
count,285.000000,285.000000,0.0,0.0,184.000000,7.000000,66.000000,30.000000,49.000000,0.0,1.0,0.0,0.0,136.000000,53.000000,9.0,2.0,285.000000,270.000000,285.000000,0.0,2.850000e+02,0.0,0.0,1.600000e+01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1221.308772,1221.308772,NaN,NaN,0.747202,0.645099,0.757289,0.798961,0.829935,NaN,1.0,NaN,NaN,2.095588,1.716981,1.0,1.0,1.782456,8.937037,757.905263,NaN,5.091186e+17,NaN,NaN,9.020191e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,238.988897,238.988897,NaN,NaN,0.132555,0.179861,0.121845,0.148648,0.122471,NaN,NaN,NaN,NaN,1.720804,1.446180,0.0,0.0,1.670490,12.138288,699.792209,NaN,2.983345e+17,NaN,NaN,2.444374e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,225.000000,225.000000,NaN,NaN,0.500718,0.500442,0.530786,0.530214,0.557950,NaN,1.0,NaN,NaN,1.000000,1.000000,1.0,1.0,1.000000,1.000000,98.000000,NaN,5.451722e+08,NaN,NaN,4.141500e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1194.000000,1194.000000,NaN,NaN,0.655807,0.504622,0.668934,0.676809,0.733641,NaN,1.0,NaN,NaN,1.000000,1.000000,1.0,1.0,1.000000,3.000000,296.000000,NaN,3.435893e+17,NaN,NaN,9.092683e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1268.000000,1268.000000,NaN,NaN,0.749693,0.567951,0.748842,0.836092,0.836844,NaN,1.0,NaN,NaN,1.000000,1.000000,1.0,1.0,1.000000,5.500000,463.000000,NaN,4.834335e+17,NaN,NaN,9.092683e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1340.000000,1340.000000,NaN,NaN,0.859718,0.760695,0.841327,0.920889,0.928648,NaN,1.0,NaN,NaN,2.250000,2.000000,1.0,1.0,2.000000,11.000000,1158.000000,NaN,6.017919e+17,NaN,NaN,2.279086e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1411.000000,1411.000000,NaN,NaN,0.991601,0.916667,0.998556,0.997153,0.999484,NaN,1.0,NaN,NaN,9.000000,7.000000,1.0,1.0,9.000000,115.000000,5585.000000,NaN,1.644454e+18,NaN,NaN,7.163811e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
regression_data.replace(np.nan,0 , inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [0]:
regression_data

,likeCount,anger,fear,joy,sadness,analytical,confident,tentative,emotion_anger,emotion_contempt,emotion_disgust,emotion_fear,emotion_happiness,emotion_neutral,emotion_sadness,emotion_surprise
0,1980,0.0,0.0,0.679337,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1980,0.0,0.0,0.679337,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,7.0,1.0,0.0
2,2282,0.0,0.0,0.909215,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
3,2282,0.0,0.0,0.909215,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,716,0.0,0.0,0.000000,0.000000,0.760380,0.530214,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,716,0.0,0.0,0.000000,0.000000,0.760380,0.530214,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,956,0.0,0.0,0.704959,0.000000,0.000000,0.000000,0.829677,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,956,0.0,0.0,0.704959,0.000000,0.000000,0.000000,0.829677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,858,0.0,0.0,0.664676,0.000000,0.619540,0.838624,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,858,0.0,0.0,0.664676,0.000000,0.619540,0.838624,0.000000,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0


In [0]:
y=regression_data.pop('likeCount')
data=regression_data

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
vif.index= data.columns
data = sm.add_constant(data.astype(float))
data.drop(columns=['const'],inplace=True)
model = sm.OLS(y.astype(float), data).fit()
# est2 = est.fit()
df_coeffs = pd.DataFrame({"Coefficients": model.params,"p": model.pvalues, 'vif':vif["VIF Factor"]})
df_coeffs_sorted = df_coeffs.reindex(df_coeffs["p"].sort_values().index)
df_coeffs_sorted.style.bar(subset = ["Coefficients", "p", "features"], align='mid', color=['#d65f5f', '#5fba7d'])

/usr/local/lib/python3.6/dist-packages/statsmodels/regression/linear_model.py:1638: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value enco

,Coefficients,p,vif
joy,621.039,5.74094e-11,1.59887
tentative,614.175,1.32558e-05,1.20281
analytical,525.841,0.000139673,1.30924
emotion_happiness,81.177,0.00274465,1.30936
emotion_neutral,131.715,0.0132416,1.34626
fear,-2.85392e-13,0.191445,nan
emotion_surprise,-611.262,0.251389,1.02974
emotion_fear,2.369e-13,0.257114,nan
emotion_anger,5.85546e-14,0.477262,nan
confident,122.874,0.488628,1.13128
